# Making predictions from a DataShop dataset
<p style="margin:30px">
    <img width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
</p>

In this tutorial, we show how to predict whether a student will succesfully answer a problem using a dataset from [CMU DataShop](https://pslcdatashop.web.cmu.edu/). While online courses are logistically efficient, the structure can make it more difficult for a teacher to understand how students are learning in their class. To try to fill in those gaps, we can apply machine learning. The workflow given here can be used to quickly **organize** and **make predictions** about any column from any similarly structured DataShop dataset. 

*If you're running this notebook yourself, please download the [geometry dataset](https://pslcdatashop.web.cmu.edu/DatasetInfo?datasetId=76) into the `data` folder in this repository. You will only need the `.txt` file. The infrastructure in this notebook will work with **any** LeanLab dataset, but you will need to change the file name in the first cell.*

## Highlights
* Show how to import a DataShop dataset into featuretools
* Show how to make custom primitives for stacking
* Show efficacy of automatic feature generation with these datasets

# Step 1: Creating a useful dataset structure
At the beginning of any project, it is worthwhile to take a moment to think about how your dataset is structured.

In these datasets the unique events come from `transactions`: places where a student interacts with a system. However, the columns of those transactions have variables that can be grouped together. 

For instance, there are only 59 distinct students for the 6778 transactions we in the geometry dataset. Those students log in to the system and have individual sessions. We can break down problems and problem steps in a similar way.

We create an entityset structure using the `datashop_to_entityset` function in [utilities](utilities.py). If you're interested in how `datashop_to_entityset` is structured, there's an associated notebook [entityset_function](entityset_function.ipynb) which explains choices made in more detail.

In [1]:
# Note that each branch is a one -> many relationship

# schools       students     problems
#        \        |         /
#   classes   sessions   problem steps
#          \     |       /
#           transactions  -- attempts
#

import utils

filename = 'data/ds2174_tx_All_Data_3991_2017_1128_123859.txt'
es = utils.datashop_to_entityset(filename)
es

Entityset: Dataset
  Entities:
    transactions (shape = [6778, 73])
    problem_steps (shape = [78, 2])
    problems (shape = [20, 1])
    sessions (shape = [59, 3])
    students (shape = [59, 2])
    ...And 3 more
  Relationships:
    transactions.Step Name -> problem_steps.Step Name
    problem_steps.Problem Name -> problems.Problem Name
    transactions.Session Id -> sessions.Session Id
    sessions.Anon Student Id -> students.Anon Student Id
    transactions.Class -> classes.Class
    ...and 2 more

Our `students` entity represents that: there are only 59 rows, one for each Anonymous student ID.

In [2]:
es['students'].df.head(3)

,Anon Student Id,first_sessions_time
Anon Student Id,,
Stu_c0bf45c22dc46067350d304ce330067e,Stu_c0bf45c22dc46067350d304ce330067e,1996-02-01 00:00:00
Stu_af3a2f63bda8c1338556108cb8d519a0,Stu_af3a2f63bda8c1338556108cb8d519a0,1996-02-01 00:00:02
Stu_d7f18a5fa205a889b0c5b0b56a7127d3,Stu_d7f18a5fa205a889b0c5b0b56a7127d3,1996-02-01 00:00:02


The transactions entity is everything that's left over after we've *normalized* out by other interesting variables.

In [3]:
es['transactions'].head(3)

,Sample Name,Transaction Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,Tutor Response Type,Tutor Response Subtype,...,CF (Factor embeddedness),CF (Factor figure-part),CF (Factor figure-type),CF (Factor non-standard-orientation-or-shape),CF (Factor parallelogram),CF (Factor parallelogram-type),CF (Factor repeat),CF (Factor required),CF (Factor trapezoid-part),End Time
Transaction Id,,,,,,,,,,,,,,,,,,,,,
499a0a18d7b6d96d4ee9c16d4bead6f2,All Data,499a0a18d7b6d96d4ee9c16d4bead6f2,GEO-408d5ed7:10e14be5d3a:-8000,1996-02-01 00:00:00,US/Eastern,0,ATTEMPT,NaN,RESULT,NaN,...,embedded,area,circle,0,0,0,initial,required,0,1996-02-01 00:00:00
d398b66148a76c537cba816efe946b85,All Data,d398b66148a76c537cba816efe946b85,GEO-408d5ed7:10e14be5d3a:-8000,1996-02-01 00:00:01,US/Eastern,1,ATTEMPT,NaN,RESULT,NaN,...,embedded,area,circle,0,0,0,initial,required,0,1996-02-01 00:00:02
1c133061306fd8e099eb0e4f2ac21430,All Data,1c133061306fd8e099eb0e4f2ac21430,GEO-408d5ed7:10e14be5d3a:-6e40,1996-02-01 00:00:02,US/Eastern,0,ATTEMPT,NaN,RESULT,NaN,...,alone,area,rectangle,0,1,rectangle,repeat,required,0,1996-02-01 00:00:02


# Step 2: Building Features

Next, we calculate a feature matrix on the `transactions` entity to try to predict the outcome of a given transaction. It's at this step that our previous setup pays off: we can automatically calculate features as if at a given point in time using Deep Feature Synthesis. Furthermore, we can guarentee that future values for `Outcome` won't be used for any calculations because we set the time index of that value to be after the cutoff time.

We use the function from utils which uses Deep Feature Synthesis with `entityset`, label and target entity defined.

In [4]:
fm, label = utils.create_features(es, label='Outcome')
print('Created {} features by encoding'.format(fm.shape[1]))
fm.head()

Building features: 792it [00:00, 2875.20it/s]
Progress: 100%|██████████| 61/61 [02:37<00:00,  2.59s/cutoff time]
Created 340 features by encoding


,Session Id = GEO-408d5ed7:10e14be5d3a:-6fb2,Session Id = GEO-408d5ed7:10e14be5d3a:-6e40,Session Id = GEO-408d5ed7:10e14be5d3a:-6a92,Session Id = GEO-408d5ed7:10e14be5d3a:-77b8,Session Id = GEO-408d5ed7:10e14be5d3a:-751b,Session Id = GEO-408d5ed7:10e14be5d3a:-60f5,Session Id = GEO-408d5ed7:10e14be5d3a:-73b1,Session Id = GEO-408d5ed7:10e14be5d3a:-7bb2,Session Id = GEO-408d5ed7:10e14be5d3a:-6bc2,Session Id = GEO-408d5ed7:10e14be5d3a:-71de,...,sessions.students.SUM(transactions.CF (Factor embedd3-tri-reg_prob_fix)),sessions.students.SUM(transactions.CF (Factor non-standard-orientation-or-shape)),sessions.students.SUM(transactions.CF (Factor parallelogram)),sessions.students.MEAN(transactions.Duration (sec)),sessions.students.MEAN(transactions.Problem View),sessions.students.MEAN(transactions.Is Last Attempt),sessions.students.MEAN(transactions.CF (Factor backward)),sessions.students.MEAN(transactions.CF (Factor embedd3-tri-reg_prob_fix)),sessions.students.MEAN(transactions.CF (Factor non-standard-orientation-or-shape)),sessions.students.MEAN(transactions.CF (Factor parallelogram))
Transaction Id,,,,,,,,,,,,,,,,,,,,,
499a0a18d7b6d96d4ee9c16d4bead6f2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1c133061306fd8e099eb0e4f2ac21430,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3ce42cc528ee77fb2db3b84c8b00ef39,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82e2897fa00c8895bea39726e8dd303a,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d398b66148a76c537cba816efe946b85,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


We have now automatically created 340 features using Deep Feature Synthesis. The next step is to make predictions using the returned feature matrix and label.

# Step 3: Making predictions
Using the feature matrix `fm_enc` and the label `label`, we find the `roc_auc_score` at five values in a time series split.

In [5]:
from sklearn.model_selection import TimeSeriesSplit

splitter = TimeSeriesSplit(n_splits=3, max_train_size=None)
utils.estimate_score(fm, label, splitter)

AUC score on time split 0 is 0.57
Feature Importances: 
1: sessions.students.MEAN(transactions.Duration (sec))
2: problem_steps.MEAN(transactions.Duration (sec))
3: problem_steps.SUM(transactions.Problem View)
4: sessions.MEAN(transactions.Duration (sec))
5: problem_steps.SUM(transactions.Duration (sec))
-----

AUC score on time split 1 is 0.56
Feature Importances: 
1: problem_steps.MEAN(transactions.Duration (sec))
2: problem_steps.SUM(transactions.Duration (sec))
3: sessions.students.MEAN(transactions.CF (Factor parallelogram))
4: attempts.MEAN(transactions.Duration (sec))
5: sessions.students.MEAN(transactions.Duration (sec))
-----

AUC score on time split 2 is 0.59
Feature Importances: 
1: problem_steps.SUM(transactions.Problem View)
2: problem_steps.SUM(transactions.Duration (sec))
3: attempts.MEAN(transactions.CF (Factor non-standard-orientation-or-shape))
4: problem_steps.MEAN(transactions.Duration (sec))
5: problem_steps.MEAN(transactions.Is Last Attempt)
-----



Notice that the feature `problem_steps.MEAN(transactions.Duration (sec))` appears in all three splits. This is the average time spent on a given problem step. It's easy to see how 'amount of time people spend on this problem' might be related to problem difficulty and ultimately the `Outcome` of a given attempt.

# Next Steps
This notebook showed how to structure your data and make predictions with machine learning. Rather than spending time creating features, it's now possible to explore the relationships and implications betweem thousands of features directly. Reasonable next steps might be to:
1. Make plots to better understand the relationship between existing features and the label 
2. Reduce the total number of features and tune the machine learning model
3. Create discipline specific *custom primitives* that might be useful for this prediction problem




# Appendix: Custom Primitives
It's often the case that you'd like to create discipline specific primitives. Here we create a custom primitive: `ProbFail`, which calculates the likelihood that a boolean variable is false. One of the many advantages in defining custom primitives is that we can define the name and input types as we would like. If you're interested in creating your own custom primitives for this dataset, copy and modify this step as necessary.

In [6]:
from featuretools.primitives import make_agg_primitive
import featuretools.variable_types as vtypes

def probability(boolean):
    numtrue = len([x for x in boolean if x==1])
    return 1 - numtrue/len(boolean)

ProbFail = make_agg_primitive(probability,
                              input_types=[vtypes.Boolean],
                              name='failure_rate',
                              description='Calculates likelihood a boolean is false over a region',
                              return_type=vtypes.Numeric)

fm2, label2 = utils.create_features(es,
                                  label='Outcome',
                                  custom_agg=[ProbFail])

utils.estimate_score(fm2, label2, splitter)


Building features: 804it [00:00, 3153.41it/s]
Progress: 100%|██████████| 61/61 [02:59<00:00,  2.95s/cutoff time]
AUC score on time split 0 is 0.55
Feature Importances: 
1: attempts.FAILURE_RATE(transactions.Outcome)
2: CF (Factor required) = additional
3: problem_steps.SUM(transactions.Duration (sec))
4: sessions.students.FAILURE_RATE(transactions.Outcome)
5: sessions.students.MEAN(transactions.Duration (sec))
-----

AUC score on time split 1 is 0.56
Feature Importances: 
1: problem_steps.FAILURE_RATE(transactions.Outcome)
2: attempts.FAILURE_RATE(transactions.Outcome)
3: problem_steps.MEAN(transactions.Duration (sec))
4: problem_steps.SUM(transactions.Duration (sec))
5: sessions.MEAN(transactions.CF (Factor backward))
-----

AUC score on time split 2 is 0.61
Feature Importances: 
1: problem_steps.MEAN(transactions.Duration (sec))
2: problem_steps.FAILURE_RATE(transactions.Outcome)
3: sessions.students.MEAN(transactions.Is Last Attempt)
4: problem_steps.SUM(transactions.Duration (sec))

In [9]:
from bokeh.io import show, output_notebook

output_notebook()
p = utils.plot(fm2,
               col1='sessions.FAILURE_RATE(transactions.Outcome)',
               col2='problem_steps.MEAN(transactions.Duration (sec))',
               label=label2,
               names=['Student success rate versus problem time', 
                      'Percentage of attempts that were mistakes by this student', 
                      'Average time it takes a student on this problem'])
#show(p)

Loading BokehJS ...

![](data/images/exampleimage.png)

Notice that *most* students have between a 10% and 30% rate of getting a problem wrong overall, and most problems take fewer than 20 seconds on average. Outside of those ranges, we can start to take educated guesses about Outcomes. 

If a student answers a problem incorrectly 40% of the time or if they problem usually takes more than 20 seconds to do, the chances are higher that the next answer will also be wrong. Similarly, if the failure rate is below 10%, chances are a lot higher the `Outcome` will be correct.

# Feature Labs
This notebook was written by [Seth Rothschild](https://www.featurelabs.com/company/) at Feature Labs. If you have questions, comments or concerns regarding the notebook feel free to [reach out](https://gitter.im/featuretools/featuretools).